In [3]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as plt

# 1. 设定参数
E = 1.0  # 弹性模量
nu = 0.3  # 泊松比
Gc = 1.0  # 断裂能
ell = 0.02  # 相场特征长度
load = 1.0  # 施加的拉伸载荷

# 2. 定义网格
L, H = 1.0, 1.0  # 计算域尺寸
nx, ny = 40, 40  # 网格划分
mesh = RectangleMesh(Point(0, 0), Point(L, H), nx, ny)

# 3. 定义有限元空间
P1 = FiniteElement("CG", mesh.ufl_cell(), 1)  # 一阶连续有限元
element = MixedElement([VectorElement("CG", mesh.ufl_cell(), 2), P1])  # 位移 + 相场
V = FunctionSpace(mesh, element)

# 4. 定义试探函数和测试函数
duphi = TrialFunction(V)
v, q = TestFunctions(V)

# 5. 设定边界条件
def left(x, on_boundary):
    return on_boundary and near(x[0], 0)

def right(x, on_boundary):
    return on_boundary and near(x[0], L)

bc_left = DirichletBC(V.sub(0), Constant((0.0, 0.0)), left)
bc_right = DirichletBC(V.sub(0).sub(0), Constant(load), right)
bcs = [bc_left, bc_right]

# 6. 定义应变和应力
def epsilon(u):
    return sym(grad(u))

def sigma(u, phi):
    return (1 - phi)**2 * 2 * E / (1 - nu**2) * epsilon(u)

# 7. 定义相场控制方程
u, phi = split(duphi)
Psi = 0.5 * inner(sigma(u, phi), epsilon(u))  # 弹性能
G = 0.5 * Gc * (phi**2 / ell + ell * dot(grad(phi), grad(phi)))  # 断裂能

F = (Psi + G) * dx  # 总能量泛函
J = derivative(F, duphi, TestFunction(V))  # 雅可比矩阵

# 8. 计算求解
duphi_ = Function(V)
solve(F == 0, duphi_, bcs, J=J)

# 9. 结果可视化
u_, phi_ = duphi_.split()
plot(phi_, title="相场变量")
plt.show()


ModuleNotFoundError: No module named 'dolfin'

In [ ]:
from dolfin import *
import numpy as np

# 1. 设置材料参数
E, nu = 1.0, 0.3  # 弹性模量, 泊松比
Gc, ell = 1.0, 0.02  # 断裂能, 相场特征长度
load_max, num_steps = 1.0, 50  # 最大载荷, 时间步数
dt = 0.01  # 时间步长

# 2. 生成自适应网格
mesh = RectangleMesh(Point(0, 0), Point(1, 1), 60, 60)
V_u = VectorFunctionSpace(mesh, "CG", 2)  # 位移场
V_phi = FunctionSpace(mesh, "CG", 1)  # 相场

# 3. 变量定义
u, phi = Function(V_u), Function(V_phi)
du, dphi = TrialFunction(V_u), TrialFunction(V_phi)
v, q = TestFunction(V_u), TestFunction(V_phi)

# 4. 应变和应力
def epsilon(u): return sym(grad(u))
def sigma(u, phi):
    return (1 - phi)**2 * 2 * E / (1 - nu**2) * epsilon(u)

# 5. 定义方程
elastic_energy = 0.5 * inner(sigma(u, phi), epsilon(u)) * dx
fracture_energy = (Gc / (2 * ell)) * (phi**2 + ell**2 * dot(grad(phi), grad(phi))) * dx
F_u = derivative(elastic_energy, u, v)
F_phi = derivative(fracture_energy - elastic_energy, phi, q)

# 6. 迭代求解
bc_u = DirichletBC(V_u.sub(0), Constant(load_max), "near(x[0], 1)")
bc_phi = DirichletBC(V_phi, Constant(0), "on_boundary")
bcs = [bc_u, bc_phi]

solver_u, solver_phi = LinearVariationalSolver(LinearVariationalProblem(lhs(F_u), rhs(F_u), u, bcs)), \
                       LinearVariationalSolver(LinearVariationalProblem(lhs(F_phi), rhs(F_phi), phi, bcs))

for step in range(num_steps):
    solver_u.solve()
    solver_phi.solve()
    plot(phi, title=f"Step {step}")



from dolfin import *
import numpy as np
import matplotlib.pyplot as plt

# 定义三维网格和有限元空间
mesh = UnitCubeMesh(20, 20, 20)
V = FunctionSpace(mesh, 'P', 1)

# 边界条件
u_L = Expression('t', degree=1, t=0.0)
def boundary(x, on_boundary):
    return on_boundary
bc = DirichletBC(V, u_L, boundary)

# 定义材料参数
E = Constant(100.0)  # 弹性模量
nu = Constant(0.3)   # 泊松比
c = Constant(10.0)   # 内聚力
phi = Constant(30.0 * pi / 180.0)  # 内摩擦角
kappa = Constant(1.0)  # 热导率
alpha = Constant(1.0e-5)  # 热膨胀系数
k_perm = Constant(1.0e-12) # 渗透系数
mu_f = Constant(1.0e-3)   # 流体粘度

# 应力-应变关系
lmbda = (E*nu)/((1+nu)*(1-2*nu))
mu = E/(2*(1+nu))

def epsilon(u):
    return sym(grad(u))

def sigma(u):
    return 2*mu*epsilon(u) + lmbda*tr(epsilon(u))*Identity(len(u))

# 摩尔库伦准则
psi = TrialFunction(V)
v = TestFunction(V)

sigma_t = sqrt(inner(sigma(psi), sigma(psi)))
friction_term = c + sigma_t * tan(phi)
G = friction_term * v * dx

# 定义相场损伤模型
alpha_d = TrialFunction(V)
v_d = TestFunction(V)
g = (1 - alpha_d)**2
w = g * inner(sigma(psi), epsilon(psi)) * dx

# 热-渗流场
T = TrialFunction(V)
v_T = TestFunction(V)
heat_flux = kappa * grad(T)
thermal_energy = inner(heat_flux, grad(T)) * dx

# 渗流场
p = TrialFunction(V)
v_p = TestFunction(V)
k_grad_p = k_perm / mu_f * grad(p)
fluid_flow = inner(k_grad_p, grad(p)) * dx

# 损伤演化方程
psi_new = Function(V)
T_new = Function(V)
p_new = Function(V)
t = 0.0
dt = 0.1
T_end = 5.0

# 时间步进求解
while t < T_end:
    t += dt
    u_L.t = t

    # 求解力学场
    solve(lhs(G) == rhs(G), psi_new, bc)

    # 更新相场损伤
    solve(lhs(w) == rhs(w), alpha_d)

    # 求解热场
    solve(lhs(thermal_energy) == rhs(thermal_energy), T_new)

    # 求解渗流场
    solve(lhs(fluid_flow) == rhs(fluid_flow), p_new)

    # 可视化
    damage = project(1 - exp(-psi_new), V)
    plot(damage, title=f'Time: {t:.1f}')

plt.show()
